# CSE 1015 - J - Component
# Fire detection

Team Members 

19BAI1035 - S.O.Narendran\
19BAI1160 - Barath Hari Hara Moorthy

# Fire Detection using Webcam

We will be using the help of the previously saved models and opencv library to capture the video and send the frames within the video to the model which will then predict the presence of a flame.

In the snippet below, we are creating a inception v3 model which will load the weights that we previously trained

In [1]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout

def modelCreation():
    input_tensor = Input(shape=(224, 224, 3))

    base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)

    # add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.2)(x)
    predictions = Dense(2, activation='softmax')(x)

    model1 = Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers:
        layer.trainable = False

    model1.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
    
    return(model1)

In [3]:
import os

checkpoint_path = "training_incep2/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

Loading weights

In [4]:
newModel = modelCreation()

# Loads the weights

newModel.load_weights(checkpoint_path)

# Video capturing

In the snippet below, we will import the open cv library which will be used to capture the video from the webcam. The frames from the video will then be converted to the RGB 

In [9]:
import cv2
import numpy as np
from PIL import Image
import tensorflow as tf
from keras.preprocessing import image
#Load the saved model
model = newModel
video = cv2.VideoCapture(0)
while True:
        _, frame = video.read()
        #Convert the captured frame into RGB
        im = Image.fromarray(frame, 'RGB')
        
        #Resizing into 224x224 because we trained the model with this image size.
        im = im.resize((224,224))
        img_array = image.img_to_array(im)
        img_array = np.expand_dims(img_array, axis=0) / 255
        probabilities = model.predict(img_array)[0]
        #Calling the predict method on model to predict 'fire' on the image
        prediction = np.argmax(probabilities)
        
        if prediction == 0:
                frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
                print("Probability that a fire exists within the frame: ", probabilities[prediction])
        
        cv2.imshow("Capturing", frame)
        key=cv2.waitKey(1)
        if key == ord('q'):
                break
video.release()
cv2.destroyAllWindows()

Probability that a fire exists within the frame:  0.57294464
Probability that a fire exists within the frame:  0.5268721
Probability that a fire exists within the frame:  0.6408865
Probability that a fire exists within the frame:  0.5455072
Probability that a fire exists within the frame:  0.5400425
Probability that a fire exists within the frame:  0.5620088
Probability that a fire exists within the frame:  0.8453756
Probability that a fire exists within the frame:  0.8674696
Probability that a fire exists within the frame:  0.63093066
Probability that a fire exists within the frame:  0.7695376
Probability that a fire exists within the frame:  0.62993956
Probability that a fire exists within the frame:  0.52425563
Probability that a fire exists within the frame:  0.54129606
Probability that a fire exists within the frame:  0.5831952
Probability that a fire exists within the frame:  0.7463704
Probability that a fire exists within the frame:  0.5121504
Probability that a fire exists with

Probability that a fire exists within the frame:  0.8763182
Probability that a fire exists within the frame:  0.95178324
Probability that a fire exists within the frame:  0.9207411
Probability that a fire exists within the frame:  0.91733676
Probability that a fire exists within the frame:  0.97919935
Probability that a fire exists within the frame:  0.97534
Probability that a fire exists within the frame:  0.96264833
Probability that a fire exists within the frame:  0.9270188
Probability that a fire exists within the frame:  0.9017347
Probability that a fire exists within the frame:  0.9461787
Probability that a fire exists within the frame:  0.97056717
Probability that a fire exists within the frame:  0.9855656
Probability that a fire exists within the frame:  0.99478936
Probability that a fire exists within the frame:  0.96854264
Probability that a fire exists within the frame:  0.88298655
Probability that a fire exists within the frame:  0.9323813
Probability that a fire exists wit

# Results

The model is detecting fire, the screenshots of which is given in the document. It does that with considerable accuracy wherein it doesn't detect small fires like the ones caused by a dia or a stove top but it definitely detects fire with higher intensity and volume.